In [1]:
from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps

import orjson

In [2]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))

/tmp/ipykernel_75793/2237727833.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# Connect to the Spatialite database
db_path = 'sqlite:////tmp/blog/spatialite_ray.db'
engine = create_engine(db_path, echo=False)  # Set echo=True to see SQL commands being executed

In [4]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')
    dbapi_connection.execute('SELECT InitSpatialMetaData(1);')

In [5]:
# Create the table
Base.metadata.create_all(engine)

InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


In [6]:
from tqdm import tqdm

In [7]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select * from geometries")).fetchall()

CPU times: user 3.27 s, sys: 5.37 s, total: 8.65 s
Wall time: 10.3 s


In [8]:
len(results)

1063260

In [9]:
results[0]

(1, 'tubule', b'\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\xa0Q\xe6@\x00\x00\x00\x00\x00\x9f\xd3@\x00\x00\x00\x00 x\xe6@\x00\x00\x00\x00\x00\xeb\xd3@|\x03\x00\x00\x00 ... (48619 characters truncated) ... a\xe6@\x00\x00\x00\x00\x00\x9f\xd3@\x00\x00\x00\x00\xc0a\xe6@\x00\x00\x00\x00\x00\x9f\xd3@\x00\x00\x00\x00\xa0a\xe6@\x00\x00\x00\x00\x00\x9f\xd3@\xfe')

In [10]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,AsGeoJSON(geom) from geometries")).fetchall()

CPU times: user 2min 33s, sys: 6.23 s, total: 2min 39s
Wall time: 2min 39s


In [11]:
results[0]

(1, 'tubule', '{"type":"Polygon","coordinates":[[[45837,20092],[45835.99999999999,20093],[45835,20093.99999999999],[45834,20094.99999999999],[45832.99999999999,2009 ... (22044 characters truncated) ... 45844,20092],[45842.99999999999,20092],[45842,20092],[45840.99999999999,20092],[45840,20092],[45839,20092],[45837.99999999999,20092],[45837,20092]]]}')

In [12]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,AsEWKB(geom) from geometries")).fetchall()

CPU times: user 6min 9s, sys: 6.65 s, total: 6min 15s
Wall time: 6min 15s


In [13]:
results[0]

(1, 'tubule', '010300002000000000010000009503000000000000A061E64000000000009FD340000000008061E64000000000409FD340000000006061E64000000000809FD340000000004061E640000 ... (29080 characters truncated) ... 2E64000000000009FD340000000000062E64000000000009FD34000000000E061E64000000000009FD34000000000C061E64000000000009FD34000000000A061E64000000000009FD340')

In [14]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,AsEWKT(geom) from geometries")).fetchall()

CPU times: user 2min 34s, sys: 2.07 s, total: 2min 36s
Wall time: 2min 36s


In [15]:
results[0]

(1, 'tubule', 'SRID=0;POLYGON((45837 20092,45835.99999999999 20093,45835 20093.99999999999,45834 20094.99999999999,45832.99999999999 20096,45832 20096,45830.9999999 ... (20191 characters truncated) ... 092,45845 20092,45844 20092,45842.99999999999 20092,45842 20092,45840.99999999999 20092,45840 20092,45839 20092,45837.99999999999 20092,45837 20092))')

In [16]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,ST_Centroid(geom) from geometries")).fetchall()

CPU times: user 17.6 s, sys: 1.88 s, total: 19.4 s
Wall time: 19.4 s


In [17]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,AsGeoJSON(ST_Centroid(geom)) from geometries")).fetchall()

CPU times: user 16.7 s, sys: 1.78 s, total: 18.5 s
Wall time: 18.5 s


In [18]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,AsEWKT(ST_Centroid(geom)) from geometries")).fetchall()

CPU times: user 15.4 s, sys: 1.75 s, total: 17.2 s
Wall time: 17.2 s


In [19]:
%%time
with  engine.connect() as conn:
    results=conn.execute(text("select id,name,AsEWKB(ST_Centroid(geom)) from geometries")).fetchall()

CPU times: user 17.7 s, sys: 1.19 s, total: 18.8 s
Wall time: 18.8 s
